In [12]:
import os
import json
import csv
import random
from paper_comparison.types import Table


gold_table_file = open("../../metric_validation_0_tables.jsonl")
gold_tables = []
for line in gold_table_file:
    data = json.loads(line)
    table = Table(data["tabid"], list(data["table"].keys()), data["table"])
    # Check that there aren't any tables for which we couldn't produce predictions
    if not os.path.exists(f"../../EMNLP2024FinalOutputs/gold_schema_values/{table.tabid}_with_better_values.json"):
        continue
    gold_tables.append(table)
print(len(gold_tables))

557


In [23]:
random.seed(42)
gold_tables_to_eval = random.sample(gold_tables, 100)
annotation_folder = "ValueEvaluation"
if not os.path.exists(annotation_folder):
    os.mkdir(annotation_folder)

missing_predictions = 0
na_predictions = 0
total_predictions = 0
for table in gold_tables_to_eval:
    pred_file = f"../../EMNLP2024FinalOutputs/gold_schema_values/{table.tabid}_with_better_values.json"
    pred_data = json.load(open(pred_file))
    pred_table = Table(pred_data["tabid"], list(pred_data["table"].keys()), pred_data["table"])
    annotation_file = open(os.path.join(annotation_folder, f"{table.tabid}_value_eval.csv"), "w")
    writer = csv.writer(annotation_file)
    writer.writerow(["Table ID", "Column", "Corpus ID", "Gold Value", "Predicted Value"])
    for column in table.schema:
        for paper in table.values[column]:
            total_predictions += 1
            if str(paper) not in pred_table.values[column]:
                missing_predictions += 1
                continue
            if pred_table.values[column][str(paper)] == "N/A":
                na_predictions += 1
            new_row = [table.tabid, column, paper, ' | '.join(table.values[column][paper]), pred_table.values[column][str(paper)]]
            writer.writerow(new_row)
    annotation_file.close()
print(f"{missing_predictions} out of {total_predictions} cells did not have predicted values")
print(f"{na_predictions} out of {total_predictions} cells have N/A predictions")

24 out of 1489 cells did not have predicted values
132 out of 1489 cells have N/A predictions
